### Import Library

In [41]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

### Read the dataset of csv file

In [42]:
df_train = pd.read_csv('BankChurners_fd5.csv')
df_train

,CustomerId,France,Germany,Spain,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditLevel
0,15599054,0.000000,0.801215,0.198785,0.740364,0.464691,0.000000,0,1,0.435955,0,3
1,15775433,0.000000,1.000000,0.000000,0.100000,0.211294,0.333333,1,1,0.561170,0,6
2,15660125,0.239005,0.760995,0.000000,0.671702,0.562711,0.333333,1,1,0.170473,0,2
3,15784304,0.369095,0.000000,0.630905,0.536910,0.352936,0.000000,1,1,0.434634,0,3
4,15642311,0.160870,0.839130,0.000000,0.548261,0.356035,0.333333,0,0,0.695248,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
2527,15630106,0.000000,0.000000,1.000000,0.200000,0.000000,0.333333,1,0,0.276947,0,3
2528,15633757,1.000000,0.000000,0.000000,0.164313,0.557549,0.196406,1,1,0.502076,0,8
2529,15798254,0.514966,0.000000,0.485034,0.600000,0.180963,0.000000,1,0,0.814625,1,0
2530,15690297,0.000000,1.000000,0.000000,0.688253,0.517317,0.215696,1,0,0.455584,1,1


#### Split the data into train and test set

In [43]:
### Change the train  file path for different users data
train = pd.read_csv('BankChurners_fdtrain.csv')
test = pd.read_csv('BankChurners_fdtest.csv')

train_y = pd.DataFrame(train['CreditLevel'])
train_X = train.drop(columns=['CreditLevel'])
train_X = train_X.drop(columns=['CustomerId'])

test_y = pd.DataFrame(test['CreditLevel'])
test_X = test.drop(columns=['CreditLevel'])
test_X = test_X.drop(columns=['CustomerId'])

In [44]:
train_set = torch.utils.data.TensorDataset(torch.Tensor(np.array(train_X)), torch.squeeze(torch.Tensor(np.array(train_y))))
test_set = torch.utils.data.TensorDataset(torch.Tensor(np.array(test_X)), torch.squeeze(torch.Tensor(np.array(test_y))))
            

train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=32, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=32, 
                                          shuffle=False)

### Network Architecture

In [45]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.01)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.01)
        x = F.softmax(self.fc3(x))
        return x

#### Loss function and learning rate

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  

### Train

In [47]:
total_step = len(train_loader)
num_epochs = 50

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        inputs = inputs.to(device)
        labels = torch.tensor(labels,dtype = torch.long)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 50 == 0:
            model.eval()
            correct = 0
            total = 0
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()),
                  'Validation Accuracy: {} %'.format(100 * correct / total))



/tmp/ipykernel_1119/3075316110.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)
/tmp/ipykernel_1119/2053517580.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


Epoch [1/50], Step [50/80], Loss: 2.2992 Validation Accuracy: 15.679304897314376 %
Epoch [2/50], Step [50/80], Loss: 2.2841 Validation Accuracy: 14.454976303317535 %
Epoch [3/50], Step [50/80], Loss: 2.2343 Validation Accuracy: 20.85308056872038 %
Epoch [4/50], Step [50/80], Loss: 2.2143 Validation Accuracy: 20.89257503949447 %
Epoch [5/50], Step [50/80], Loss: 2.2447 Validation Accuracy: 21.12954186413902 %
Epoch [6/50], Step [50/80], Loss: 2.2523 Validation Accuracy: 21.011058451816744 %
Epoch [7/50], Step [50/80], Loss: 2.2778 Validation Accuracy: 22.748815165876778 %
Epoch [8/50], Step [50/80], Loss: 2.2065 Validation Accuracy: 23.143759873617693 %
Epoch [9/50], Step [50/80], Loss: 2.1728 Validation Accuracy: 22.82780410742496 %
Epoch [10/50], Step [50/80], Loss: 2.2472 Validation Accuracy: 22.86729857819905 %
Epoch [11/50], Step [50/80], Loss: 2.2005 Validation Accuracy: 22.669826224328595 %
Epoch [12/50], Step [50/80], Loss: 2.1726 Validation Accuracy: 23.30173775671406 %
Epoch [

### Model evalulation

In [8]:
model.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print(predicted)

print('Accuracy of the network on the 1000 data: {} %'.format(100 * correct / total))

tensor([6, 6, 5, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 5, 6, 6, 5, 6, 6, 5, 6,
        6, 6, 5, 6, 6, 6, 5, 6, 5, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 5,
        6, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 5, 6, 5, 5, 6], device='cuda:0')
tensor([6, 6, 5, 6, 5, 6, 5, 5, 6, 5, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5, 5,
        5, 6, 5, 6, 6, 6, 6, 5, 5, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6,
        6, 6, 6, 5, 6, 6, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6], device='cuda:0')
tensor([5, 6, 6, 6, 5, 5, 6, 6, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6, 5, 6, 6, 5, 6, 6,
        6, 6, 5, 6, 6, 5, 5, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5,
        6, 6, 6, 6, 6, 5, 6, 5, 6, 6, 5, 5, 6, 6, 5, 6], device='cuda:0')
tensor([6, 6, 6, 6, 5, 6, 6, 6, 6, 5, 5, 6, 6, 5, 6, 5, 6, 6, 5, 5, 6, 6, 5, 5,
        6, 5, 5, 6, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 6,
        5, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 5], device='cuda:0')
tensor([6, 5, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 6, 6, 5, 5, 5, 6, 6,

/tmp/ipykernel_817/2053517580.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc3(x))


### Save the model

In [9]:
torch.save(model,'save.pt')

In [10]:
torch.load('save.pt')

NN(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)

--END--